In [1]:
# Use the endpoint POST Create session to create a query session for flights.
# Required: originPlace: SFO-sky
#           destinationPlace: 
#           outboundDate:
    
#     Get back the session ID in the Location header from POST Create session.
#     Poll the results using the endpoint GET Poll session results.

In [ ]:
import sky as sk
import pandas as pd
import requests

sky_api_host = 'https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/'

In [16]:
# POST to initiate Session request

post_url = sky_api_host + "/apiservices/pricing/v1.0"

payload = "inboundDate=&cabinClass=&children=&infants=&country=&currency=&locale=&originPlace=&destinationPlace=&outboundDate=&adults="
headers = {
    'x-rapidapi-host': sky_api_host,
    'x-rapidapi-key': sk.sky_key,
    'content-type': "application/x-www-form-urlencoded"
    }

response = requests.request("POST", post_url, data=payload, headers=headers)

print(response.text)

In [ ]:

arguments = ''
# url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsequotes/v1.0/US/USD/en-US/SFO-sky/JFK-sky/2019-09-01"
# querystring = {"inboundpartialdate":"2019-12-01"}

headers = {
    'x-rapidapi-host': sky_url,
    'x-rapidapi-key': sky_key
    }

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)